In [3]:
from carla import MLModelCatalog, MLModel
from carla.recourse_methods import GrowingSpheres
from carla.data.catalog import OnlineCatalog

from carla.data.catalog import CsvCatalog


In [2]:
data_name = 'adult'

dataset = OnlineCatalog(data_name)


model = MLModelCatalog(dataset,"ann")

hyperparameters = {}

gs = GrowingSpheres(model, hyperparameters)


factuals = dataset.df.sample(10)

counterfactuals = gs.get_counterfactuals(factuals)

counterfactuals

[WARNING] From /home/swang8/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]
[WARNING] From /home/swang8/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]


/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.205479,0.276246,0.663674,0.134692,-0.007893,...,0.0,1.0,0.0,0.0,1.0
1,0.000000,0.074598,0.465062,0.131335,-0.003193,...,1.0,1.0,1.0,1.0,1.0
2,0.328767,0.219759,0.569630,0.097578,0.117519,...,0.0,1.0,1.0,1.0,1.0
3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
4,0.068493,0.214874,0.639796,0.153643,-0.052295,...,0.0,1.0,1.0,0.0,1.0
5,0.068493,0.206935,0.538408,0.052441,-0.026430,...,0.0,1.0,0.0,0.0,1.0
6,0.506849,0.237417,0.541465,0.037743,0.019937,...,1.0,0.0,0.0,1.0,0.0
7,0.095890,0.118212,0.838959,0.037615,-0.003601,...,0.0,1.0,1.0,0.0,1.0
8,0.630137,0.117613,0.529505,0.010839,0.003395,...,0.0,1.0,0.0,0.0,0.0
9,0.068493,0.136297,0.646758,0.071218,0.012245,...,1.0,1.0,1.0,0.0,1.0


In [7]:
 
continuous = ["age", "fnlwgt", "education-num", "capital-gain","hours-per-week", "capital-loss"]
categorical = ["marital-status", "native-country", "occupation", "race", "relationship", "sex", "workclass"]
immutable = ["age", "sex"]

dataset = CsvCatalog(file_path="adult.csv",
                     continuous=continuous,
                     categorical=categorical,
                    immutables=immutable,
                   target='income')



In [9]:
from carla.models.catalog import MLModelCatalog

training_params = {"lr": 0.002, "epochs": 10, "batch_size": 1024, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    backend="pytorch"
)

ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"]
)


Loaded model from /home/swang8/carla/models/custom/ann_layers_18_9_3.pt
test accuracy for model: 0.8348623853211009


/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [10]:
# recourse 



from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

display(test_factual)



hyperparams = {"loss_type": "BCE", "binary_cat_features": True}
recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.021740,0.0,...,0.0,1.0,1.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.000000,0.0,...,1.0,0.0,0.0,1.0,1.0
6,0.438356,0.100061,0.266667,0.000000,0.0,...,1.0,0.0,1.0,0.0,1.0
8,0.191781,0.022661,0.866667,0.140841,0.0,...,0.0,1.0,1.0,0.0,1.0


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,age,capital-gain,capital-loss,education-num,fnlwgt,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.350855,0.070798,0.048407,0.848995,-0.004223,...,0.0,1.0,1.0,1.0,0.0
2,0.402153,0.115434,0.116777,0.648935,0.018383,...,1.0,1.0,1.0,1.0,1.0
3,0.571809,0.078703,0.078683,0.478694,0.072046,...,1.0,0.0,0.0,1.0,1.0
6,0.658569,0.223400,0.222516,0.487981,-0.083927,...,1.0,0.0,1.0,0.0,1.0
8,0.201781,0.150841,0.010000,0.876667,0.012661,...,0.0,1.0,1.0,0.0,1.0


In [11]:
hyperparams = {
    "data_name": dataset.name,
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "clamp": True,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [len(ml_model.feature_input_order), 512, 256, 8],
        "train": True,
        "lambda_reg": 1e-6,
        "epochs": 5,
        "lr": 1e-3,
        "batch_size": 32,
    },
}

cchvae = recourse_catalog.CCHVAE(ml_model, hyperparams)
df_cfs = cchvae.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Start training of Variational Autoencoder... [models.py fit]
[INFO] [Epoch: 0/5] [objective: 0.381] [models.py fit]
[INFO] [ELBO train: 0.38] [models.py fit]
[INFO] [ELBO train: 0.14] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] ... finished training of Variational Autoencoder. [models.py fit]


/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'torch.Tensor'>
  f"cannot re-order features for non dataframe input: {type(x)}"
/home/swang8/anaconda3/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,age,capital-gain,capital-loss,education-num,fnlwgt,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.297449,0.037508,0.040037,0.605625,0.120442,...,1.0,1.0,1.0,1.0,1.0
2,0.299593,0.038349,0.039616,0.603004,0.120385,...,1.0,1.0,1.0,1.0,1.0
3,0.300599,0.038263,0.039112,0.604303,0.120417,...,1.0,1.0,1.0,1.0,1.0
6,0.300136,0.038156,0.039987,0.603068,0.120385,...,1.0,1.0,1.0,1.0,1.0
8,0.301572,0.039048,0.039432,0.602171,0.120381,...,1.0,1.0,1.0,1.0,1.0


In [12]:
ml_model = MLModelCatalog(dataset, "forest", backend="sklearn", load_online=False)
ml_model.train(max_depth=2, n_estimators=5, force_train=True)

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

display(test_factual)


balance on test set 0.23795871559633028, balance on test set 0.24344692005242463
model fitted with training score 0.8040629095674967 and test score 0.8014416775884666


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.00000,0.0,...,0.0,0.0,1.0,0.0,1.0


In [13]:

from carla import MLModel
import xgboost

class XGBoostModel(MLModel):
    """The default way of implementing XGBoost
    https://xgboost.readthedocs.io/en/latest/python/python_intro.html"""

    def __init__(self, data):
        super().__init__(data)

        # get preprocessed data
        df_train = self.data.df_train
        df_test = self.data.df_test

        x_train = df_train[self.data.continuous]
        y_train = df_train[self.data.target]
        x_test = df_test[self.data.continuous]
        y_test = df_test[self.data.target]

        self._feature_input_order = self.data.continuous

        param = {
            "max_depth": 2,  # determines how deep the tree can go
            "objective": "binary:logistic",  # determines the loss function
            "n_estimators": 5,
        }
        self._mymodel = xgboost.XGBClassifier(**param)
        self._mymodel.fit(
                x_train,
                y_train,
                eval_set=[(x_train, y_train), (x_test, y_test)],
                eval_metric="logloss",
                verbose=True,
            )

    @property
    def feature_input_order(self):
        # List of the feature order the ml model was trained on
        return self._feature_input_order

    @property
    def backend(self):
        # The ML framework the model was trained on
        return "xgboost"

    @property
    def raw_model(self):
        # The black-box model object
        return self._mymodel

    @property
    def tree_iterator(self):
        # make a copy of the trees, else feature names are not saved
        booster_it = [booster for booster in self.raw_model.get_booster()]
        # set the feature names
        for booster in booster_it:
            booster.feature_names = self.feature_input_order
        return booster_it

    # The predict function outputs
    # the continuous prediction of the model
    def predict(self, x):
        return self._mymodel.predict(self.get_ordered_features(x))

    # The predict_proba method outputs
    # the prediction as class probabilities
    def predict_proba(self, x):
        return self._mymodel.predict_proba(self.get_ordered_features(x))
    




ml_model = XGBoostModel(dataset)

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

display(test_factual)



[0]	validation_0-logloss:0.58383	validation_1-logloss:0.58389
[1]	validation_0-logloss:0.52317	validation_1-logloss:0.52375
[2]	validation_0-logloss:0.48489	validation_1-logloss:0.48587
[3]	validation_0-logloss:0.45972	validation_1-logloss:0.46122
[4]	validation_0-logloss:0.44078	validation_1-logloss:0.44290


/home/swang8/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.00000,0.0,...,0.0,0.0,1.0,0.0,1.0
